In [ ]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='6d0e9548-df2a-449b-a40b-6797fff51b56', project_access_token='p-77f21f8bdd6d17abf4b6ebdac9e63b177e7b7f44')
pc = project.project_context


In [15]:
import pandas as pd

Scrap the data frame from the Wikipedia page:

In [16]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

Drop those rows where __Borough__ is <span style="color:blue">Not assigned</span>:

In [35]:
df = df[df['Borough'] != 'Not assigned']

If __Neighborhood__ is <span style="color:blue">Not assigned</span>, then set it to __Borough__:

In [36]:
df['Neighborhood'][df['Neighborhood'] == 'Not assigned'] = df['Borough']

Where there are multiple __Neighborhood__ entries for the same __Postcode__ and __Borough__, combine them in a single record:

In [37]:
df = df.groupby(['Postcode', 'Borough'])['Neighborhood'].apply(lambda x: ', '.join(x)).reset_index()

In [38]:
df.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


So what is the number of rows and columns in the dataframe?

In [39]:
print(df.shape)

(103, 3)


Now we will try to determine the geographical coordinates of the post codes via __geocoder__...

In [11]:
#!conda install -c conda-forge geocoder --yes

In [44]:
import geocoder as gc

First, run a quick test for one post code:

In [45]:
postal_code = df['Postcode'][0]
lat_lng_coords = None
while(lat_lng_coords is None):
    g = gc.google('{}, Toronto, Ontario'.format(postal_code))
    lat_lng_coords = g.latlng
latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

KeyboardInterrupt: 

The __geocoder__ method does not seem to work, therefore we will use the workaround and load the coordinated from an external file:

In [46]:
postal_codes = pd.read_csv('http://cocl.us/Geospatial_data')

In [47]:
postal_codes.rename(columns={'Postal Code': 'Postcode'}, inplace=True)

In [48]:
postal_codes.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [49]:
new_df = pd.merge(df, postal_codes, how='right', on='Postcode')

In [50]:
new_df.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [51]:
new_df.shape

(103, 5)

In [28]:
#!conda install -c conda-forge folium=0.5.0 --yes

In [52]:
from geopy.geocoders import Nominatim
import folium

In [53]:
address = 'Toronto, Ontario, Canada'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of ' + address + ' are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Ontario, Canada are 43.653963, -79.387207.


Let's save here the coordinates of Toronto in case __geolocator__ is in a funny mood the next time we need to re-run the code:  
latitude = 43.653963  
longitude = -79.387207

Let's plot our neighborhoods:

In [70]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, borough, neighborhood in zip(new_df['Latitude'], new_df['Longitude'], new_df['Borough'], new_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

map_toronto

Let's extract data from Foursquare:

In [71]:
CLIENT_ID = 'PSOQHB3TGPK3SKK24AY1FDQTXQMKV1LDYSDQXBGMCZ2HFE0P' # Foursquare ID
CLIENT_SECRET = 'NK4C03ZOFGMKXQEJFITL1CSXUSRJ0I5O3IMMZJWLQUPWZ0K1' # Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [75]:
import requests

In [73]:
# We will proceed with radius 500 and limit 100 for Toronto, although we understand they may need to be adjusted!
def getNearbyVenues(names, latitudes, longitudes, radius=500, limit=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [120]:
venues = getNearbyVenues(names=new_df['Postcode'], latitudes=new_df['Latitude'], longitudes=new_df['Longitude'])

M1B
M1C
M1E
M1G
M1H
M1J
M1K
M1L
M1M
M1N
M1P
M1R
M1S
M1T
M1V
M1W
M1X
M2H
M2J
M2K
M2L
M2M
M2N
M2P
M2R
M3A
M3B
M3C
M3H
M3J
M3K
M3L
M3M
M3N
M4A
M4B
M4C
M4E
M4G
M4H
M4J
M4K
M4L
M4M
M4N
M4P
M4R
M4S
M4T
M4V
M4W
M4X
M4Y
M5A
M5B
M5C
M5E
M5G
M5H
M5J
M5K
M5L
M5M
M5N
M5P
M5R
M5S
M5T
M5V
M5W
M5X
M6A
M6B
M6C
M6E
M6G
M6H
M6J
M6K
M6L
M6M
M6N
M6P
M6R
M6S
M7A
M7R
M7Y
M8V
M8W
M8X
M8Y
M8Z
M9A
M9B
M9C
M9L
M9M
M9N
M9P
M9R
M9V
M9W


In [175]:
venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M1B,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Restaurant
1,M1C,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,M1E,43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,M1E,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,M1E,43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa


We should probably run some data cleaning here...

In [176]:
print('Number of unique categories: ' + str(len(venues['Venue Category'].unique())))

Number of unique categories: 224


In [177]:
print('Number of unique restaurant categories: ' + str(len(venues['Venue Category'][venues['Venue Category'].str.contains('Restaurant')].unique())))

Number of unique restaurant categories: 1


Let's collapse the restaurant categories into one, why not?

In [178]:
venues['Venue Category'][venues['Venue Category'].str.contains('Restaurant')] = 'Restaurant'

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [179]:
print('Number of unique categories: ' + str(len(venues['Venue Category'].unique())))

Number of unique categories: 224


In [180]:
venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M1B,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Restaurant
1,M1C,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,M1E,43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,M1E,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,M1E,43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa


In [181]:
venues.shape

(2209, 7)

In [182]:
venues_onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")
venues_onehot['Postcode'] = venues['Neighborhood']
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])
venues_onehot = venues_onehot[fixed_columns]
venues_onehot.head()

,Postcode,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,Antique Shop,Aquarium,...,Trail,Train Station,Video Game Store,Video Store,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M1B,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M1C,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M1E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M1E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M1E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [183]:
venues_onehot.shape

(2209, 225)

Let's group rows by post code, taking the mean of the frequency of occurence of each category

In [184]:
venues_grouped = venues_onehot.groupby('Postcode').mean().reset_index()
venues_grouped.tail(5)

,Postcode,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,Antique Shop,Aquarium,...,Trail,Train Station,Video Game Store,Video Store,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
94,M9N,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
95,M9P,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
96,M9R,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
97,M9V,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.111111,0.0,0.0,0.0,0.0,0.0,0.0
98,M9W,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [185]:
venues_grouped.shape

(99, 225)

In [186]:
import numpy as np

In [187]:
def get_most_common_venues(row, num_top_venues=10):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [203]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Postcode'] = venues_grouped['Postcode']

for ind in np.arange(venues_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = get_most_common_venues(venues_grouped.iloc[ind, :], num_top_venues)

venues_sorted.head()

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Restaurant,Yoga Studio,Deli / Bodega,Festival,Farmers Market,Event Space,Electronics Store,Drugstore,Donut Shop,Dog Run
1,M1C,Bar,Yoga Studio,Fish Market,Field,Festival,Farmers Market,Event Space,Electronics Store,Drugstore,Donut Shop
2,M1E,Intersection,Breakfast Spot,Spa,Pizza Place,Electronics Store,Medical Center,Restaurant,Rental Car Location,Yoga Studio,Dessert Shop
3,M1G,Coffee Shop,Restaurant,Insurance Office,Yoga Studio,Department Store,Festival,Farmers Market,Event Space,Electronics Store,Drugstore
4,M1H,Restaurant,Fried Chicken Joint,Athletics & Sports,Bakery,Gas Station,Bank,Donut Shop,Dog Run,Discount Store,Deli / Bodega


In [204]:
from sklearn.cluster import KMeans

In [205]:
kclusters = 5

venues_grouped_clustering = venues_grouped.drop('Postcode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(venues_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 3, 0, 4, 4, 0, 0, 0, 4, 0], dtype=int32)

Add clustering labels and merge the original data frame with the clustered result

In [206]:
venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
venues_merged = new_df
venues_merged = venues_merged.join(venues_sorted.set_index('Postcode'), on='Postcode')

float64


In [219]:
venues_merged.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,4.0,Restaurant,Yoga Studio,Deli / Bodega,Festival,Farmers Market,Event Space,Electronics Store,Drugstore,Donut Shop,Dog Run
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,3.0,Bar,Yoga Studio,Fish Market,Field,Festival,Farmers Market,Event Space,Electronics Store,Drugstore,Donut Shop
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0.0,Intersection,Breakfast Spot,Spa,Pizza Place,Electronics Store,Medical Center,Restaurant,Rental Car Location,Yoga Studio,Dessert Shop
3,M1G,Scarborough,Woburn,43.770992,-79.216917,4.0,Coffee Shop,Restaurant,Insurance Office,Yoga Studio,Department Store,Festival,Farmers Market,Event Space,Electronics Store,Drugstore
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,4.0,Restaurant,Fried Chicken Joint,Athletics & Sports,Bakery,Gas Station,Bank,Donut Shop,Dog Run,Discount Store,Deli / Bodega


Check if there are such neighborhoods for which Foursquare did not return a single venue - these therefore have been excluded from clustering

In [230]:
venues_merged[venues_merged['Cluster Labels'].isna()]

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,M1X,Scarborough,Upper Rouge,43.836125,-79.205636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93,M9A,Queen's Park,Queen's Park,43.667856,-79.532242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94,M9B,Etobicoke,"Cloverdale, Islington, Martin Grove, Princess ...",43.650943,-79.554724,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


There are 4: M1X, M2M, M9A, M9B  
Looking at Google Maps, there are actually quite a few venues in the respective boroughs  
Perhaps we should have readjusted the coordinates of these post codes and/or amended the radius of the Foursquare search

For the sake of visualisation, let us drop these four neighborhoods:

In [232]:
venues_merged = venues_merged.drop(venues_merged[venues_merged['Cluster Labels'].isna()].index)

In [237]:
venues_merged['Cluster Labels'] = venues_merged['Cluster Labels'].astype('int32')
venues_merged.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,4,Restaurant,Yoga Studio,Deli / Bodega,Festival,Farmers Market,Event Space,Electronics Store,Drugstore,Donut Shop,Dog Run
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,3,Bar,Yoga Studio,Fish Market,Field,Festival,Farmers Market,Event Space,Electronics Store,Drugstore,Donut Shop
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0,Intersection,Breakfast Spot,Spa,Pizza Place,Electronics Store,Medical Center,Restaurant,Rental Car Location,Yoga Studio,Dessert Shop
3,M1G,Scarborough,Woburn,43.770992,-79.216917,4,Coffee Shop,Restaurant,Insurance Office,Yoga Studio,Department Store,Festival,Farmers Market,Event Space,Electronics Store,Drugstore
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,4,Restaurant,Fried Chicken Joint,Athletics & Sports,Bakery,Gas Station,Bank,Donut Shop,Dog Run,Discount Store,Deli / Bodega


In [238]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [239]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(venues_merged['Latitude'], venues_merged['Longitude'], venues_merged['Postcode'], venues_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Cluster 0

In [240]:
venues_merged.loc[venues_merged['Cluster Labels'] == 0, venues_merged.columns[[1] + list(range(5, venues_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,0,Intersection,Breakfast Spot,Spa,Pizza Place,Electronics Store,Medical Center,Restaurant,Rental Car Location,Yoga Studio,Dessert Shop
5,Scarborough,0,Playground,Grocery Store,Dance Studio,Farmers Market,Event Space,Electronics Store,Drugstore,Donut Shop,Dog Run,Discount Store
6,Scarborough,0,Discount Store,Coffee Shop,Hobby Shop,Department Store,Yoga Studio,Deli / Bodega,Festival,Farmers Market,Event Space,Electronics Store
7,Scarborough,0,Bus Line,Bakery,Park,Restaurant,Metro Station,Intersection,Bus Station,Soccer Field,Coworking Space,Concert Hall
9,Scarborough,0,College Stadium,Skating Rink,General Entertainment,Café,Yoga Studio,Deli / Bodega,Farmers Market,Event Space,Electronics Store,Drugstore
20,North York,0,Martial Arts Dojo,Cafeteria,Department Store,Field,Festival,Farmers Market,Event Space,Electronics Store,Drugstore,Donut Shop
24,North York,0,Pizza Place,Grocery Store,Discount Store,Butcher,Pharmacy,Coffee Shop,College Stadium,Department Store,Farmers Market,Event Space
31,North York,0,Bank,Grocery Store,Park,Hotel,Yoga Studio,Festival,Farmers Market,Event Space,Electronics Store,Drugstore
32,North York,0,Home Service,Food Truck,Baseball Field,Yoga Studio,Department Store,Festival,Farmers Market,Event Space,Electronics Store,Drugstore
33,North York,0,Coffee Shop,Grocery Store,Liquor Store,Athletics & Sports,Yoga Studio,Department Store,Festival,Farmers Market,Event Space,Electronics Store


#### Cluster 1

In [241]:
venues_merged.loc[venues_merged['Cluster Labels'] == 1, venues_merged.columns[[1] + list(range(5, venues_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,Etobicoke,1,Baseball Field,Yoga Studio,Fish Market,Field,Festival,Farmers Market,Event Space,Electronics Store,Drugstore,Donut Shop
97,North York,1,Construction & Landscaping,Baseball Field,Yoga Studio,Dessert Shop,Field,Festival,Farmers Market,Event Space,Electronics Store,Drugstore


#### Cluster 2

In [242]:
venues_merged.loc[venues_merged['Cluster Labels'] == 2, venues_merged.columns[[1] + list(range(5, venues_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,2,Playground,Park,Yoga Studio,Dance Studio,Farmers Market,Event Space,Electronics Store,Drugstore,Donut Shop,Dog Run
23,North York,2,Park,Convenience Store,Bank,Yoga Studio,Department Store,Festival,Farmers Market,Event Space,Electronics Store,Drugstore
25,North York,2,Park,Food & Drink Shop,Yoga Studio,Deli / Bodega,Farmers Market,Event Space,Electronics Store,Drugstore,Donut Shop,Dog Run
30,North York,2,Airport,Park,Yoga Studio,Deli / Bodega,Festival,Farmers Market,Event Space,Electronics Store,Drugstore,Donut Shop
40,East York,2,Park,Convenience Store,Yoga Studio,Deli / Bodega,Festival,Farmers Market,Event Space,Electronics Store,Drugstore,Donut Shop
48,Central Toronto,2,Playground,Park,Tennis Court,Yoga Studio,Dance Studio,Farmers Market,Event Space,Electronics Store,Drugstore,Donut Shop
50,Downtown Toronto,2,Park,Playground,Trail,Yoga Studio,Dance Studio,Farmers Market,Event Space,Electronics Store,Drugstore,Donut Shop
74,York,2,Park,Market,Women's Store,Restaurant,Deli / Bodega,Festival,Farmers Market,Event Space,Electronics Store,Drugstore
90,Etobicoke,2,Park,Smoke Shop,River,Yoga Studio,Dance Studio,Event Space,Electronics Store,Drugstore,Donut Shop,Dog Run
98,York,2,Park,Convenience Store,Yoga Studio,Deli / Bodega,Festival,Farmers Market,Event Space,Electronics Store,Drugstore,Donut Shop


#### Cluster 3

In [243]:
venues_merged.loc[venues_merged['Cluster Labels'] == 3, venues_merged.columns[[1] + list(range(5, venues_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,3,Bar,Yoga Studio,Fish Market,Field,Festival,Farmers Market,Event Space,Electronics Store,Drugstore,Donut Shop


#### Cluster 4

In [244]:
venues_merged.loc[venues_merged['Cluster Labels'] == 4, venues_merged.columns[[1] + list(range(5, venues_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,4,Restaurant,Yoga Studio,Deli / Bodega,Festival,Farmers Market,Event Space,Electronics Store,Drugstore,Donut Shop,Dog Run
3,Scarborough,4,Coffee Shop,Restaurant,Insurance Office,Yoga Studio,Department Store,Festival,Farmers Market,Event Space,Electronics Store,Drugstore
4,Scarborough,4,Restaurant,Fried Chicken Joint,Athletics & Sports,Bakery,Gas Station,Bank,Donut Shop,Dog Run,Discount Store,Deli / Bodega
8,Scarborough,4,Skating Rink,Motel,Restaurant,Movie Theater,Yoga Studio,Deli / Bodega,Event Space,Electronics Store,Drugstore,Donut Shop
10,Scarborough,4,Restaurant,Pet Store,Light Rail Station,Yoga Studio,Deli / Bodega,Farmers Market,Event Space,Electronics Store,Drugstore,Donut Shop
11,Scarborough,4,Breakfast Spot,Smoke Shop,Restaurant,Bakery,Yoga Studio,Department Store,Farmers Market,Event Space,Electronics Store,Drugstore
12,Scarborough,4,Lounge,Breakfast Spot,Clothing Store,Skating Rink,Restaurant,Department Store,Farmers Market,Event Space,Electronics Store,Drugstore
13,Scarborough,4,Restaurant,Pharmacy,Pizza Place,Bank,Noodle House,Convenience Store,Fried Chicken Joint,Shopping Mall,Discount Store,Dog Run
15,Scarborough,4,Restaurant,Coffee Shop,Thrift / Vintage Store,Pharmacy,Pizza Place,Nail Salon,Sandwich Place,Breakfast Spot,Electronics Store,Supermarket
17,North York,4,Restaurant,Athletics & Sports,Golf Course,Pool,Dog Run,Yoga Studio,Discount Store,Department Store,Dessert Shop,Diner
